In [423]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn import preprocessing

DATA_DIR = "../input"

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'

CHUNKSIZE = 50
NROWS = 100

TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)
TRAIN_CATEGORICAL = "{0}/train_categorical.csv".format(DATA_DIR)
TRAIN_DATE = "{0}/train_date.csv".format(DATA_DIR)

TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)
TEST_CATEGORICAL = "{0}/test_categorical.csv".format(DATA_DIR)
TEST_DATE = "{0}/test_date.csv".format(DATA_DIR)

train = pd.read_csv(TRAIN_NUMERIC, usecols=[ID_COLUMN, TARGET_COLUMN], nrows=NROWS)
test = pd.read_csv(TEST_NUMERIC, usecols=[ID_COLUMN], nrows=NROWS)

In [424]:
# Preprocessing categorical data

# Load data
train_df1 = pd.read_csv(TRAIN_CATEGORICAL, nrows=NROWS)
test_df1 = pd.read_csv(TEST_CATEGORICAL, nrows=NROWS)

# Merge into single DataFrame such that we get consistent dummies at train and test
merged = pd.concat([train_df1, test_df1])

# Preprocess
merged.fillna(0, inplace=True)
merged = pd.get_dummies(merged)

# Load back
train_df1 = merged[:train_df1.shape[0]]
test_df1 = merged[train_df1.shape[0]:]

In [425]:
# Preprocessing remaining train data

train_df2 = pd.read_csv(TRAIN_DATE, nrows=NROWS)
train_df2.fillna(0, inplace=True)
train_df2.ix[:,1:] = min_max_scaler.fit_transform(train_df2.ix[:,1:])

train_df3 = pd.read_csv(TRAIN_NUMERIC, nrows=NROWS)
train_df3.fillna(0, inplace=True)
train_df3.ix[:,1:] = min_max_scaler.fit_transform(train_df3.ix[:,1:])

# Merge DataFrames
train_df = pd.merge(pd.merge(train_df1, train_df2, on='Id'), train_df3, on='Id')
# Drop Id as we don't need it anymore
train_df = train_df.drop("Id", 1)
# Fill NAs
train_df.fillna(0, inplace=True)

In [426]:
# Preprocessing test data

test_df2 = pd.read_csv(TEST_DATE, nrows=NROWS)
test_df2.fillna(0, inplace=True)
test_df2.ix[:,1:] = min_max_scaler.fit_transform(test_df2.ix[:,1:])

test_df3 = pd.read_csv(TEST_NUMERIC, nrows=NROWS)
test_df3.fillna(0, inplace=True)
test_df3.ix[:,1:] = min_max_scaler.fit_transform(test_df3.ix[:,1:])

# Merge DataFrames
test_df = pd.merge(pd.merge(test_df1, test_df2, on='Id'), test_df3, on='Id')
id_test = test['Id'].values
# Drop Id as we don't need it anymore
test_df = test_df.drop("Id", 1)
# Fill NAs
test_df.fillna(0, inplace=True)

In [427]:
# Extract feature (X) and target (y) columns
feature_cols = list(train_df.columns[:-1])  # all columns but last are features
target_col = train_df.columns[-1]  # last column is the target/label
X_all = train_df[feature_cols]  # feature values for all products
y_all = train_df[target_col]  # corresponding targets
print('Feature and target columns separated.')

Feature and target columns separated.


In [428]:
# Select features (X) and corresponding labels (y) for the training and validation sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size = 0.85, random_state=42)

# Convert data to Numpy matrices for Keras
X_train = np.asmatrix(X_train)
X_test = np.asmatrix(X_test)
test_df = np.asmatrix(test_df)

print "Training set: {} samples".format(X_train.shape[0])
print "Validation set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 85 samples
Validation set: 15 samples


In [429]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU

def build_model():
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], init = 'he_normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(32, init = 'he_normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam')
    return(model)

In [431]:
nb_epoch = 1
batch_size = 16

model = build_model()
fit = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch = nb_epoch, verbose = 2 ,validation_data=[X_test, y_test])
pred = model.predict_classes(test_df, batch_size=batch_size)
    
print('Exporting final result...')
df = pd.DataFrame({'Id': id_test})
df['Response'] = pred
df.to_csv('../output/submission_keras.csv', index = False)

print('Results ready for submission.')

Train on 85 samples, validate on 15 samples
Epoch 1/1
0s - loss: 2.2100 - val_loss: 1.1921e-07
 96/100 [===========================>..] - ETA: 0sExporting final result...
Results ready for submission.


In [413]:
nb_epoch = 1
batch_size = 16
nb_bags = 1
pred_test = np.zeros(test_df.shape[0])

for i in range(nb_bags):
    print('nb_bags: {}').format(i)
    model = build_model()
    fit = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch = nb_epoch, verbose = 2 ,validation_data=[X_test, y_test])
    pred_test += model.predict_classes(test_df, batch_size=batch_size)
    
## test predictions
print('Exporting final result...')
pred_test /= (nfolds*nbags)
df = pd.DataFrame({'id': id_test, 'loss': pred_test})
df.to_csv('submission_keras_bagged.csv', index = False)

print('Ready for submission.')

nb_bags: 0
Train on 85 samples, validate on 15 samples
Epoch 1/1
0s - loss: 0.2908 - val_loss: 1.1921e-07
100/100 [==============================] - 0s     
Exporting final result...


Exception: Data must be 1-dimensional

In [339]:
from sklearn.cross_validation import KFold

nfolds = 5
nepochs = 1
nbags = 1
folds = KFold(X_train.shape[1], n_folds = nfolds, shuffle = True, random_state = 1)
#folds = KFold(15, n_folds = nfolds, shuffle = True, random_state = 1)

for (inTr, inTe) in folds:
    xtr = X_train[inTr]
    ytr = y_train[inTr]
    xte = X_train[inTe]
    yte = y_train[inTe]
    pred = np.zeros(xte.shape[0])
    for j in range(nbags):
        print('Nbags: {}').format(j)
        model = build_model()
        fit = model.fit(xtr, ytr, batch_size=1, nb_epoch = nepochs, verbose = 1, validation_data=[xte, yte])
        pred_test += model.predict_classes(test_df, batch_size=800)
    i += 1
    print('Fold: ', i)
    
## test predictions
print('Exporting final result...')
pred_test /= (nfolds*nbags)
df = pd.DataFrame({'id': id_test, 'loss': pred_test})
df.to_csv('submission_keras_bagged.csv', index = False)

print('Ready for submission.')

IndexError: index 85 is out of bounds for axis 0 with size 85